In [1]:
import pandas as pd
import scipy.stats as stats

# https://github.com/isaac-michel224/Healthcare-Data-Analytics---Data-Mining/blob/main/project3.ipynb

In [2]:
df = pd.read_csv('puf2016.csv', low_memory=False)
df.head(10)

,PUF_ID,SVYYR,ADM_H_ENT,ADM_H_MEDSTA,ADM_H_GHPSW,ADM_H_PDRS,ADM_OP_MDCD,ADM_PARTD,ADM_ACOFLAG,ADM_FFS_FLAG_YR,...,EEYRS091,EEYRS092,EEYRS093,EEYRS094,EEYRS095,EEYRS096,EEYRS097,EEYRS098,EEYRS099,EEYRS100
0,16000001,2016,2,3,2,1.0,2,1,1,3,...,1730.863696,315.422808,1772.439746,2014.649096,325.924565,1515.141757,313.119607,2046.638105,393.058043,380.735043
1,16000002,2016,1,1,2,1.0,1,2,2,3,...,6448.706281,24524.935702,4837.843981,7129.014867,29151.731944,29396.561818,5072.077989,27137.197856,5097.901657,4878.055919
2,16000006,2016,2,1,1,1.0,1,1,2,1,...,3589.669228,499.620974,517.477715,2904.884066,2866.824292,580.906592,535.377989,3361.963502,3010.362589,497.151719
3,16000008,2016,2,1,1,1.0,1,1,2,1,...,3740.417968,783.611754,4256.555251,824.458231,4553.515534,665.623951,767.555640,680.624207,3780.704442,671.883483
4,16000012,2016,2,1,2,1.0,1,1,2,3,...,4104.799801,749.123388,3521.030891,713.109543,3600.575591,705.059726,3831.571491,650.836074,4093.686322,768.398950
5,16000013,2016,2,1,1,1.0,1,1,2,1,...,581.538823,628.516429,3325.480654,683.222446,3245.491288,666.556473,3268.595057,3496.216067,620.991980,729.444157
6,16000014,2016,2,1,1,1.0,1,1,2,1,...,5712.638646,5805.219030,4996.156765,5303.779755,5641.298384,1076.000113,1173.059553,1056.656768,1104.498216,5685.821448
7,16000015,2016,2,1,1,1.0,1,1,2,1,...,632.937838,609.323613,3195.927639,644.221073,3760.514308,3386.390780,633.731772,636.272074,3000.938546,3083.339718
8,16000019,2016,2,1,2,1.0,1,2,1,3,...,749.483088,830.309858,625.901655,658.253306,666.387931,4665.916717,3892.379144,4684.827926,4443.823949,3570.560655
9,16000021,2016,2,1,2,1.0,3,1,2,3,...,631.358983,3191.678566,3772.474440,3650.307068,3744.032269,3067.077863,3581.101861,3315.881984,3558.086527,3648.861429


In [3]:
dfq1 = df.loc[(df['ADM_H_MEDSTA'] == 1) & (df['DEM_RACE'].isin([1, 2])) & (df['ACC_HCDELAY'].isin(['1', '2'])), ['DEM_RACE','ACC_HCDELAY']].reset_index()
dfq1.columns = ['index', 'race', 'financial_difficulty']
dfq1['race'] = dfq1['race'].map({1:'Non-Hispanic White', 2:'Non-Hispanic Black'})
dfq1['financial_difficulty'] = dfq1['financial_difficulty'].map({'1':'with financial difficulty', '2':"without financial difficulty"})
dfq1_grp = dfq1.groupby(['race', 'financial_difficulty']).count().unstack()
dfq1_grp.columns =dfq1_grp.columns.droplevel(0)
writer = pd.ExcelWriter('project3.xlsx')
dfq1_grp.to_excel(writer, sheet_name='q1', header=True)
dfq1_grp

financial_difficulty,with financial difficulty,without financial difficulty
race,,
Non-Hispanic Black,77,772
Non-Hispanic White,462,7731


In [4]:
#Calculate p-value for Question 1 - Statistically Significant

_, pvalue = stats.fisher_exact([dfq1_grp.iloc[0,:], dfq1_grp.iloc[1,:]], alternative='greater')
pvalue

9.710017237578831e-05

In [5]:
#Question 2

In [6]:
dfq2 = df.loc[df['ADM_H_MEDSTA'] == 1, ['DEM_SEX', 'ADM_H_PHYEVT']]
dfq2.columns = ['gender', 'utilization']
mapping = {0:0, 1:3, 2:8, 3:13, 4:18, 5:23}
dfq2['utilization'] = dfq2['utilization'].map(mapping)
dfq2['gender'] = dfq2['gender'].map({1:'male', 2:'female'})
dfq2.head()

,gender,utilization
1,male,0
2,male,0
3,male,0
4,male,8
5,male,0


In [7]:
dfq2_grp = dfq2.groupby('gender').mean()
dfq2_grp = dfq2_grp.applymap(lambda x: round(x, 2))
dfq2_grp.to_excel(writer, 'q2')
dfq2_grp

,utilization
gender,
female,5.26
male,4.89


In [8]:
filt = (dfq2['gender'] == 'male')
stats.ttest_ind(dfq2.loc[filt]['utilization'], dfq2.loc[~filt]
               ['utilization'], equal_var=True)

Ttest_indResult(statistic=-2.8886720891111812, pvalue=0.0038765385147276313)

In [9]:
dfq3 = df.loc[(df['DEM_EDU'].isin(['1', '2', '3'])), ['HLT_BMI_CAT', 'DEM_EDU']].reset_index()
dfq3.columns = ['index', 'is_obesity', 'educations']
dfq3 = dfq3.dropna()
dfq3['is_obesity'] = ['obese' if x > 3 else 'healthy' for x in dfq3['is_obesity']]
dfq3['educations'] = ['high education' if y == '3' else 'low education' for y in dfq3['educations']]
dfq3.head()

,index,is_obesity,educations
1,1,obese,high education
2,2,healthy,low education
3,3,obese,low education
4,4,healthy,low education
5,5,healthy,low education


In [10]:
dfq3_grp = dfq3.groupby(['educations', 'is_obesity']).count().unstack()
dfq3_grp.columns = dfq3_grp.columns.droplevel(0)
dfq3_grp.to_excel(writer, 'q3')
dfq3_grp


is_obesity,healthy,obese
educations,,
high education,4014,1591
low education,4599,2208


In [11]:
_, pvalue = stats.fisher_exact([dfq3_grp.iloc[0,:], dfq3_grp.iloc[1,:]], alternative='greater')
pvalue

5.818209055554241e-07

In [12]:
#Question 4

In [13]:
dfq4 = df.loc[(df['HLT_OCDEPRSS'].isin(['1','2'])), ['HLT_BMI_CAT','HLT_OCDEPRSS']]
dfq4 = dfq4.dropna()
dfq4.columns = ['is_obese', 'is_depressed']
dfq4['is_obese'] = ['obese' if x > 3 else 'healthy' for x in dfq4['is_obese']]
dfq4['is_depressed'] = ['is depressed' if x == '1' else 'not depressed' for x in dfq4['is_depressed']]
dfq4 = dfq4.reset_index()
dfq4.head()

,index,is_obese,is_depressed
0,1,obese,not depressed
1,2,healthy,not depressed
2,3,obese,not depressed
3,4,healthy,not depressed
4,5,healthy,not depressed


In [14]:
dfq4_grp = dfq4.groupby(['is_obese', 'is_depressed']).count().unstack()
dfq4_grp.columns = dfq4_grp.columns.droplevel(0)
dfq4_grp.to_excel(writer, 'q4')
dfq4_grp

is_depressed,is depressed,not depressed
is_obese,,
healthy,2039,6593
obese,1366,2446


In [16]:
_, pvalue = stats.fisher_exact([dfq4_grp.iloc[0,:], dfq4_grp.iloc[1,:]], alternative='less')
pvalue

4.892727337268204e-44

In [17]:
#Question 5

In [20]:
male = df.loc[(df['HLT_OCDEPRSS'].isin(['1', '2'])) & (df['DEM_SEX'] == 1), 
              ['HLT_BMI_CAT', 'HLT_OCDEPRSS']]
male = male.dropna()
male.columns = ['male_is_obese', 'male_is_depressed']
male['male_is_obese'] = ['obese' if x > 3 else 'healthy' for x in male['male_is_obese']]
male['male_is_depressed'] = ['is depressed' if x == '1' else 'not depressed' for x in male['male_is_depressed']]
male = male.reset_index()
male.head()

,index,male_is_obese,male_is_depressed
0,1,obese,not depressed
1,2,healthy,not depressed
2,3,obese,not depressed
3,4,healthy,not depressed
4,5,healthy,not depressed


In [24]:
female = df.loc[(df['HLT_OCDEPRSS'].isin(['1','2'])) & (df['DEM_SEX'] == 2), ['HLT_BMI_CAT', 'HLT_OCDEPRSS']]
female = female.dropna()
female.columns = ['female_is_obese', 'female_is_depressed']
female['female_is_obese'] = ['obese' if x > 3 else 'healthy' for x in female['female_is_obese']]
female['female_is_depressed'] = ['is depressed' if x == '1' else 'not depressed' for x in female['female_is_depressed']]
female = female.reset_index()
female.head()

,index,female_is_obese,female_is_depressed
0,7,healthy,not depressed
1,8,healthy,not depressed
2,9,healthy,not depressed
3,11,obese,is depressed
4,12,obese,not depressed


In [25]:
dfq5_male = male.groupby(['male_is_obese', 'male_is_depressed']).count().unstack()
dfq5_male.columns = dfq5_male.columns.droplevel(0)
dfq5_male.to_excel(writer, 'q5_male')
dfq5_male

male_is_depressed,is depressed,not depressed
male_is_obese,,
healthy,779,3236
obese,491,1189


In [26]:
dfq5_female = female.groupby(['female_is_obese', 'female_is_depressed']).count().unstack()
dfq5_female.columns = dfq5_female.columns.droplevel(0)
dfq5_female.to_excel(writer, 'q5_female')
dfq5_female

female_is_depressed,is depressed,not depressed
female_is_obese,,
healthy,1260,3357
obese,875,1257


In [27]:
_, male_pvalue = stats.fisher_exact([dfq5_male.iloc[0,:], dfq5_male.iloc[1,:]], alternative='less')
male_pvalue

9.39942851821692e-16

In [28]:
_, female_pvalue = stats.fisher_exact([dfq5_female.iloc[0,:], dfq5_female.iloc[1,:]], alternative='less')
female_pvalue

3.785303010447029e-29

In [29]:
#Question 6

In [33]:
dfq6 = df.loc[(df['ADM_H_MEDSTA'] == 1) & (df['HLT_GENHELTH'].isin(['1', '2', '3', '4', '5'])) &
             (df['DEM_MARSTA'].isin(['1', '2', '3', '4'])), ['HLT_GENHELTH', 'DEM_MARSTA']]
dfq6.columns = ['general health', 'loneliness']
dfq6['general health'] = ['Poor Fair Health' if x in ['4','5']
                         else 'Good Health' for x in dfq6['general health']]
dfq6['loneliness'] = ['With Family' if x == '1' else 'Living Alone' for x in dfq6['loneliness']]
dfq6 = dfq6.reset_index()
dfq6.head()

,index,general health,loneliness
0,1,Good Health,With Family
1,2,Good Health,Living Alone
2,3,Good Health,With Family
3,4,Good Health,Living Alone
4,5,Good Health,With Family


In [34]:
dfq6_grp = dfq6.groupby(['loneliness', 'general health']).count().unstack()
dfq6_grp.columns = dfq6_grp.columns.droplevel(0)
dfq6_grp.to_excel(writer,'q6')
dfq6_grp

general health,Good Health,Poor Fair Health
loneliness,,
Living Alone,4033,1049
With Family,4590,895


In [35]:
_, pvalue = stats.fisher_exact([dfq6_grp.iloc[0,:], dfq6_grp.iloc[1,:]], alternative='less')
pvalue

5.789098545095839e-09

In [36]:
#Question 7

In [37]:
dfq7 = df.loc[(df['ADM_H_MEDSTA'] == 1) & (df['HLT_OCDEPRSS'].isin(['1', '2'])) & (df['DEM_MARSTA'].isin(['1', '2', '3', '4'])), ['HLT_OCDEPRSS','DEM_MARSTA']]
dfq7.columns = ['is_depressed', 'loneliness']
dfq7['is_depressed'] = ['is depressed' if x == '1' else 'not depressed' for x in dfq7['is_depressed']]
dfq7['loneliness'] = ['With Family' if x == '1' else 'Living Alone' for x in dfq7['loneliness']]
dfq7 = dfq7.reset_index()
dfq7.head()

,index,is_depressed,loneliness
0,1,not depressed,With Family
1,2,not depressed,Living Alone
2,3,not depressed,With Family
3,4,not depressed,Living Alone
4,5,not depressed,With Family


In [39]:
dfq7_grp = dfq7.groupby(['loneliness', 'is_depressed']).count().unstack()
dfq7_grp.columns = dfq7_grp.columns.droplevel(0)
dfq7_grp.to_excel(writer, 'q7')
dfq7_grp

is_depressed,is depressed,not depressed
loneliness,,
Living Alone,1255,3839
With Family,992,4513


In [40]:
_, pvalue = stats.fisher_exact([dfq7_grp.iloc[0,:], dfq7_grp.iloc[1,:]], alternative='greater')
pvalue

5.009099064981693e-17

In [41]:
writer.save()